## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-17-20-49-02 +0000,nyt,Trump Administration Live Updates: European Le...,https://www.nytimes.com/live/2025/08/17/us/tru...
1,2025-08-17-20-47-03 +0000,nyt,Israelis Join Nationwide Rallies to Call for R...,https://www.nytimes.com/2025/08/17/world/middl...
2,2025-08-17-20-34-10 +0000,nyt,Zelensky Brings Backup to the White House as T...,https://www.nytimes.com/2025/08/17/us/politics...
3,2025-08-17-20-31-47 +0000,latimes,16-year-old boy fatally shot at Covina house p...,https://www.latimes.com/california/story/2025-...
4,2025-08-17-20-28-09 +0000,nypost,Mayor Adams slams Mamdani for wanting to decri...,https://nypost.com/2025/08/17/us-news/mayor-ad...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2376/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,30
20,putin,18
51,ukraine,16
265,war,9
7,zelensky,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
136,2025-08-17-01-10-58 +0000,nypost,First Lady Melania Trump calls for end to war ...,https://nypost.com/2025/08/16/world-news/first...,94
162,2025-08-16-21-22-09 +0000,nyt,Trump Backs Off Cease-Fire Demand in Ukraine W...,https://www.nytimes.com/2025/08/16/world/europ...,86
158,2025-08-16-21-40-53 +0000,wapo,"Trump drops ceasefire demand for Ukraine war, ...",https://www.washingtonpost.com/world/2025/08/1...,83
86,2025-08-17-12-18-22 +0000,nypost,"European leaders, NATO boss will back up Zelen...",https://nypost.com/2025/08/17/world-news/europ...,78
68,2025-08-17-15-34-03 +0000,nypost,More than 700 National Guard troops from 3 GOP...,https://nypost.com/2025/08/17/us-news/more-tha...,73


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
136,94,2025-08-17-01-10-58 +0000,nypost,First Lady Melania Trump calls for end to war ...,https://nypost.com/2025/08/16/world-news/first...
86,48,2025-08-17-12-18-22 +0000,nypost,"European leaders, NATO boss will back up Zelen...",https://nypost.com/2025/08/17/world-news/europ...
68,35,2025-08-17-15-34-03 +0000,nypost,More than 700 National Guard troops from 3 GOP...,https://nypost.com/2025/08/17/us-news/more-tha...
61,31,2025-08-17-16-22-13 +0000,nypost,LI teen allegedly attacked by rival’s mom with...,https://nypost.com/2025/08/17/us-news/long-isl...
38,29,2025-08-17-18-52-45 +0000,nypost,Nationwide strike erupts across Israel demandi...,https://nypost.com/2025/08/17/world-news/natio...
101,24,2025-08-17-10-00-00 +0000,latimes,News Analysis: Newsom's decision to fight fire...,https://www.latimes.com/california/story/2025-...
73,24,2025-08-17-14-37-47 +0000,nypost,HUD launches English-only initiative for all d...,https://nypost.com/2025/08/17/us-news/hud-laun...
75,23,2025-08-17-14-21-19 +0000,nypost,Disgraced ex-Rep. Anthony Weiner makes blunt p...,https://nypost.com/2025/08/17/us-news/disgrace...
22,22,2025-08-17-19-36-42 +0000,nypost,Lindsey Graham threatens bill designating Russ...,https://nypost.com/2025/08/17/us-news/lindsey-...
126,21,2025-08-17-04-41-06 +0000,nypost,Ukraine ‘ghost’ sniper claims world’s longest ...,https://nypost.com/2025/08/17/world-news/ukrai...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
